# BIOSTAT707 Group Project

RNN/LSTM Model

Developed by: Jackson Dial and Caitlyn Nguyen

The following code is adapted from:
https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [1]:
# Library import
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Set Seed
tf.random.set_seed(1)

In [3]:
# All training dataset
all_train_df = pd.read_csv("train.csv").iloc[: , 1:]
all_train_df.head(5)

,subjid,Time,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,...,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,height_cleaned,MechVent_cleaned,in_hosp_death
0,132547,06:36,NaN,45.0,47.0,64,NaN,15.0,NaN,212.0,...,35.8,1.3,NaN,140.0,NaN,114.0,7.29,180.3,1,0
1,132547,06:38,NaN,45.0,47.0,64,NaN,15.0,NaN,212.0,...,35.8,1.3,NaN,140.0,NaN,114.0,7.29,180.3,1,0
2,132547,06:53,NaN,45.0,47.0,64,NaN,15.0,NaN,212.0,...,35.8,1.3,NaN,400.0,NaN,114.0,7.29,180.3,1,0
3,132547,07:23,NaN,45.0,47.0,64,NaN,15.0,NaN,212.0,...,35.8,1.3,NaN,400.0,24.0,114.0,7.29,180.3,1,0
4,132547,07:53,NaN,45.0,47.0,64,NaN,15.0,NaN,212.0,...,35.8,1.3,NaN,400.0,24.0,114.0,7.29,180.3,1,0


In [4]:
# Testing dataset
test_df = pd.read_csv("test.csv").iloc[: , 1:]
test_df.head(5)

,subjid,Time,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,...,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,height_cleaned,MechVent_cleaned,in_hosp_death
0,132540,01:11,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,...,35.2,NaN,NaN,NaN,7.4,76.0,7.45,175.3,1,0
1,132540,01:26,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,...,35.1,NaN,NaN,770.0,7.4,76.0,7.45,175.3,1,0
2,132540,01:27,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,...,35.1,NaN,NaN,0.0,7.4,76.0,7.45,175.3,1,0
3,132540,01:31,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,...,34.8,NaN,NaN,0.0,7.4,76.0,7.45,175.3,1,0
4,132540,01:38,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,...,34.8,NaN,NaN,0.0,7.4,76.0,7.44,175.3,1,0


In [5]:
# Concat df
df = pd.concat([all_train_df, test_df])
len(df["subjid"].unique())

1815

In [6]:
# Scale continuous vars
nonscaled_cols = ["subjid", "Time", "Gender", "ICUType", "in_hosp_death", "MechVent_cleaned"]
all_cols = list(df.columns.values)
scaled_cols = set(all_cols) - set(nonscaled_cols)
clean_df = df.drop(nonscaled_cols, axis = 1)
# Normalize the dataset
scaled_df = (clean_df-clean_df.min())/(clean_df.max()-clean_df.min())
clean_df = pd.concat((df[nonscaled_cols], scaled_df), 1)
# Change dtypes
clean_df.fillna(-1)
clean_df ["in_hosp_death"] = clean_df["in_hosp_death"].astype('category')
clean_df["Gender"] = clean_df["Gender"].astype('category')
clean_df["ICUType"] = clean_df["ICUType"].astype('category')
clean_df["MechVent_cleaned"] = clean_df["MechVent_cleaned"].astype('category')
clean_df.dtypes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  


subjid                 int64
Time                  object
Gender              category
ICUType             category
in_hosp_death       category
MechVent_cleaned    category
ALP                  float64
ALT                  float64
AST                  float64
Age                  float64
Albumin              float64
BUN                  float64
Bilirubin            float64
Cholesterol          float64
Creatinine           float64
DiasABP              float64
FiO2                 float64
GCS                  float64
Glucose              float64
HCO3                 float64
HCT                  float64
HR                   float64
K                    float64
Lactate              float64
MAP                  float64
MechVent             float64
Mg                   float64
NIDiasABP            float64
NIMAP                float64
NISysABP             float64
Na                   float64
PaCO2                float64
PaO2                 float64
Platelets            float64
RespRate      

In [7]:
# Get train dataset
new_train = clean_df[clean_df["subjid"].isin(all_train_df["subjid"].unique())]
len(new_train["subjid"].unique())

1270

In [8]:
# Get test dataset
new_test = clean_df[clean_df["subjid"].isin(test_df["subjid"].unique())]
len(new_test["subjid"].unique())

545

In [9]:
# Validation dataset
all_train_ids = new_train["subjid"].unique()
n = len(all_train_ids)
n_85 = math.ceil(n*0.85)
train_ids = random.sample(set(all_train_ids), n_85)
val_ids = set(all_train_ids) - set(train_ids)

val_df = new_train[new_train["subjid"].isin(val_ids)]
print(len(val_df["subjid"].unique()))
val_df.head(5)

190


,subjid,Time,Gender,ICUType,in_hosp_death,MechVent_cleaned,ALP,ALT,AST,Age,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,height_cleaned
1149,132694,02:33,0,1,1,0,NaN,NaN,NaN,0.60274,...,NaN,0.369718,0.413793,NaN,NaN,0.000000,NaN,0.651347,0.585366,0.576372
1150,132694,03:18,0,1,1,0,NaN,NaN,NaN,0.60274,...,NaN,0.369718,0.413793,NaN,NaN,0.000000,NaN,0.651347,0.585366,0.576372
1151,132694,03:23,0,1,1,0,NaN,NaN,NaN,0.60274,...,NaN,0.362676,0.448276,NaN,NaN,0.000364,NaN,0.651347,0.585366,0.576372
1152,132694,03:33,0,1,1,0,NaN,NaN,NaN,0.60274,...,NaN,0.362676,0.448276,NaN,NaN,0.000364,NaN,0.651347,0.585366,0.576372
1153,132694,04:23,0,1,1,0,NaN,NaN,NaN,0.60274,...,NaN,0.390845,0.448276,NaN,NaN,0.001636,NaN,0.651347,0.585366,0.576372


In [10]:
train_df = new_train[new_train["subjid"].isin(train_ids)]
print(len(train_df["subjid"].unique()))
train_df.head(5)

1080


,subjid,Time,Gender,ICUType,in_hosp_death,MechVent_cleaned,ALP,ALT,AST,Age,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,height_cleaned
0,132547,06:36,1,1,0,1,NaN,0.004203,0.002552,0.643836,...,NaN,0.528169,0.37931,0.02045,NaN,0.012727,NaN,0.449287,0.512195,0.696897
1,132547,06:38,1,1,0,1,NaN,0.004203,0.002552,0.643836,...,NaN,0.535211,0.37931,0.02045,NaN,0.012727,NaN,0.449287,0.512195,0.696897
2,132547,06:53,1,1,0,1,NaN,0.004203,0.002552,0.643836,...,NaN,0.426056,0.37931,0.02045,NaN,0.036364,NaN,0.449287,0.512195,0.696897
3,132547,07:23,1,1,0,1,NaN,0.004203,0.002552,0.643836,...,NaN,0.426056,0.37931,0.02045,NaN,0.036364,0.153698,0.449287,0.512195,0.696897
4,132547,07:53,1,1,0,1,NaN,0.004203,0.002552,0.643836,...,NaN,0.422535,0.37931,0.02045,NaN,0.036364,0.153698,0.449287,0.512195,0.696897


In [11]:
# Dataset matrix
def create_dataset(dataset):
  subjids = dataset["subjid"].unique()
  data = []
  bad_ids = []
  for subjid in subjids:
    ds = dataset[dataset["subjid"] == subjid].iloc[0:3, 2:]
    if ds.shape[0] < 3:
      bad_ids.append(subjid)
      continue
    else:
      data.append(np.array(ds))
  return data, bad_ids

In [12]:
train_df_x = train_df.copy().drop("in_hosp_death", axis = 1)
train_df_x["Gender"] = train_df_x["Gender"].astype('category')
train_df_x["ICUType"] = train_df_x["ICUType"].astype('category')
train_df_x["MechVent_cleaned"] = train_df_x["MechVent_cleaned"].astype('category')
train_x, bad_ids = create_dataset(train_df_x)
train_x = tf.stack(train_x)
train_x

<tf.Tensor: shape=(1076, 3, 42), dtype=float64, numpy=
array([[[1.        , 1.        , 1.        , ..., 0.44928685,
         0.51219512, 0.69689737],
        [1.        , 1.        , 1.        , ..., 0.44928685,
         0.51219512, 0.69689737],
        [1.        , 1.        , 1.        , ..., 0.44928685,
         0.51219512, 0.69689737]],

       [[0.        , 3.        , 0.        , ..., 0.18938193,
         0.64634146, 0.48568019],
        [0.        , 3.        , 0.        , ..., 0.18938193,
         0.64634146, 0.48568019],
        [0.        , 3.        , 0.        , ..., 0.18938193,
         0.64634146, 0.48568019]],

       [[0.        , 2.        , 1.        , ..., 0.21949287,
         0.69512195, 0.424821  ],
        [0.        , 2.        , 1.        , ..., 0.21949287,
         0.69512195, 0.424821  ],
        [0.        , 2.        , 1.        , ..., 0.21949287,
         0.69512195, 0.424821  ]],

       ...,

       [[0.        , 2.        , 1.        , ..., 0.22622821,


In [18]:
val_x = val_df.copy().drop("in_hosp_death", axis = 1)
val_x["Gender"] = val_x["Gender"].astype('category')
val_x["ICUType"] = val_x["ICUType"].astype('category')
val_x["MechVent_cleaned"] = val_x["MechVent_cleaned"].astype('category')
val_x, bad_ids = create_dataset(val_x)
val_x = tf.stack(val_x)
val_x

<tf.Tensor: shape=(188, 3, 42), dtype=float64, numpy=
array([[[0.        , 1.        , 0.        , ..., 0.65134707,
         0.58536585, 0.57637232],
        [0.        , 1.        , 0.        , ..., 0.65134707,
         0.58536585, 0.57637232],
        [0.        , 1.        , 0.        , ..., 0.65134707,
         0.58536585, 0.57637232]],

       [[1.        , 4.        , 0.        , ..., 0.28605388,
         0.68292683, 0.51551313],
        [1.        , 4.        , 0.        , ..., 0.28605388,
         0.68292683, 0.51551313],
        [1.        , 4.        , 0.        , ..., 0.28605388,
         0.68292683, 0.51551313]],

       [[1.        , 2.        , 1.        , ..., 0.3973851 ,
         0.62195122, 0.66706444],
        [1.        , 2.        , 1.        , ..., 0.3973851 ,
         0.62195122, 0.66706444],
        [1.        , 2.        , 1.        , ..., 0.3973851 ,
         0.62195122, 0.66706444]],

       ...,

       [[1.        , 2.        , 1.        , ..., 0.29239303,
 

In [13]:
train_y = train_df[~train_df["subjid"].isin(bad_ids)].groupby('subjid')["in_hosp_death"].first()
train_y = tf.stack(train_y)
train_y

<tf.Tensor: shape=(1076,), dtype=int64, numpy=array([0, 1, 0, ..., 0, 0, 0])>

In [14]:
# Create the RNN
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 42)))
model.add(Dense(1))

optimizer = optimizers.Adam(clipvalue=0.5)

model.compile(optimizer=optimizer, loss = tf.keras.losses.BinaryCrossentropy(from_logits=False))

In [15]:
model.fit(train_x, train_y, epochs=100, validation_split=0.2, verbose=1)

Epoch 1/100
27/27 [==============================] - 3s 18ms/step - loss: nan - val_loss: nan
Epoch 2/100
27/27 [==============================] - 0s 5ms/step - loss: nan - val_loss: nan
Epoch 3/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 4/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 5/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 6/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 7/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 8/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 9/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 10/100
27/27 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 11/100
27/27 [==============================] - 0s 6ms/step - 

In [16]:
model.evaluate(train_x)

34/34 [==============================] - 1s 5ms/step - loss: 0.0000e+00


0.0